# Example: Profit Diagram and Breakeven Points for a Short Strangle
A short strangle involves selling a `call` and `put` option on the same underlying stock, where both legs have the same expiration date. This example will familiarize students with computing the payoff, profit, and breakeven points for a short strangle at expiration.

## Learning objectives
Fill me in

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-133/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites
We'll build our short strangle using options chain data for `62-DTE` contracts on [Micron Technology](https://www.micron.com/about), which has the ticker symbol `MU.` 
* Load the options chain dataset by calling the `MyOptionsChainDataSet()` function, which returns the pricing data as a `DataFrame` instance, which is exported by the [DataFrames.jl package](https://github.com/JuliaData/DataFrames.jl). Let's save the options data in the `dataset::DataFrame` variable:

In [2]:
dataset = MyOptionsChainDataSet()

Row,Strike,Moneyness,Bid,Mid,Ask,Last,Volume,Open Int,Delta,IV,Type
,Float64,Float64,Float64,Float64,Float64,Float64,String7,String7,Float64,Float64,String7
1,32.5,0.5912,46.9,47.22,47.55,49.16,0,2,0.99881,0.6458,Call
2,35.0,0.5597,44.4,44.8,45.2,39.75,0,1,0.99364,0.8463,Call
3,37.5,0.5283,41.95,42.33,42.7,0.0,0,0,0.99303,0.784,Call
4,40.0,0.4969,39.5,39.9,40.3,46.57,0,24,0.98822,0.7904,Call
5,42.5,0.4654,36.9,37.35,37.8,43.15,0,1,0.99424,0.6253,Call
6,45.0,0.434,34.55,34.88,35.2,37.85,0,27,0.9936,0.5769,Call
7,47.5,0.4025,32.15,32.5,32.85,30.35,0,85,0.98231,0.6363,Call
8,50.0,0.3711,29.65,30.03,30.4,30.35,2,65,0.95655,0.7286,Call
9,55.0,0.3082,24.85,25.2,25.55,25.69,30,377,0.96381,0.5384,Call


## Task 1: Select the strike prices for the `call` and `put` legs of the strangle
Let the `put` leg of the stangle have index `1`, and the `call` leg have index `2`. For a stangle, the strike price of the `put` leg $K_{1}$ is less than the strike price of the `call` leg $K_{2}$, but the current share price $S_{\circ}$ is in between the strikes, i.e., $K_{1}<S_{\circ}<K_{2}$.
* The share price of `MU` is $S_{o}$ = `79.50 USD/share` for this options chain. Set this value in the `Sₒ` variable:

In [3]:
Sₒ = 79.50; # USD/share

Now, let's select random strike prices for the `put` and `call` legs such that the inequality is true. Filter `dataset` to find the `put` contracts with $K_{1}<S_{\circ}$. To do this, we'll use the [filter function](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) exported by the [DataFrames.jl package](https://github.com/JuliaData/DataFrames.jl) in combination with the [Julia pipe operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping)

In [4]:
put_contracts = filter(:Type=>x->x == "Put", dataset) |> y -> filter(:Strike=> x-> x<Sₒ, y)

Row,Strike,Moneyness,Bid,Mid,Ask,Last,Volume,Open Int,Delta,IV,Type
,Float64,Float64,Float64,Float64,Float64,Float64,String7,String7,Float64,Float64,String7
1,32.5,-0.5912,0.0,0.03,0.05,0.04,0,3,0.0,0.0,Put
2,35.0,-0.5597,0.0,0.01,0.02,0.14,0,35,0.0,0.0,Put
3,37.5,-0.5283,0.0,0.07,0.14,0.01,0,420,0.0,0.0,Put
4,40.0,-0.4969,0.01,0.02,0.03,0.04,0,238,-0.00292,0.6422,Put
5,42.5,-0.4654,0.02,0.1,0.17,0.01,1,197,-0.01077,0.7152,Put
6,45.0,-0.434,0.01,0.05,0.09,0.07,0,344,-0.00716,0.602,Put
7,47.5,-0.4025,0.05,0.08,0.11,0.08,15,"3,191",-0.0112,0.5867,Put
8,50.0,-0.3711,0.11,0.13,0.14,0.1,2,815,-0.01708,0.572,Put
9,55.0,-0.3082,0.2,0.22,0.23,0.21,11,"4,332",-0.02986,0.5137,Put


Select a random contract from the `put_contracts` collection; this will be our first leg of the strangle. Save this data in the `put_leg_data` variable, which is type [DataFrameRow](https://dataframes.juliadata.org/stable/lib/types/#DataFrames.DataFrameRow):

In [5]:
number_of_put_contracts = nrow(put_contracts)
put_leg_data = put_contracts[rand(1:number_of_put_contracts),:]

Row,Strike,Moneyness,Bid,Mid,Ask,Last,Volume,Open Int,Delta,IV,Type
,Float64,Float64,Float64,Float64,Float64,Float64,String7,String7,Float64,Float64,String7
5,42.5,-0.4654,0.02,0.1,0.17,0.01,1,197,-0.01077,0.7152,Put


Do the same procedure, but now for the `call` contracts; save the data for the random `call` contract in the `call_leg_data` variable (which is also type  type [DataFrameRow](https://dataframes.juliadata.org/stable/lib/types/#DataFrames.DataFrameRow)):

In [6]:
call_contracts = filter(:Type=>x->x == "Call", dataset) |> y -> filter(:Strike=> x-> x > Sₒ, y)
number_of_call_contracts = nrow(call_contracts)
call_leg_data = call_contracts[rand(1:number_of_call_contracts),:]

Row,Strike,Moneyness,Bid,Mid,Ask,Last,Volume,Open Int,Delta,IV,Type
,Float64,Float64,Float64,Float64,Float64,Float64,String7,String7,Float64,Float64,String7
19,150.0,-0.8868,0.0,0.03,0.06,0.01,72,22,0.00214,0.513,Call


## Task 2: Build contract models and compute the payoff, profit, and breakeven values
Now that we have data for the contracts, we can compute the composite payoff, profit, and breakeven values for the short strange constructed from the randomly selected `put` and `call` legs.

* First, create `MyAmericanPutContractModel` and `MyAmericanCallContractModel` models, initialized using the data from the random contracts from `Task 1`. Then, these contract models and possible share prices at expiration are passed to the `payoff(...)` function, which computes the individual and composite payoff and profit values.

Let's start with the randomly selected `put` contract. 

In [7]:
put_contract_model = build(MyAmericanPutContractModel, (
        K = put_leg_data[:Strike], 
        premium = put_leg_data[:Mid],
        sense = -1, # we are short, i.e., we *sold* this contract so sense = -1 
        copy = 1));

Similarly, we construct the `call` contract model.

In [8]:
call_contract_model = build(MyAmericanCallContractModel, (
        K = call_leg_data[:Strike], 
        premium = call_leg_data[:Mid],
        sense = -1, # we are short, i.e., we *sold* this contract so sense = -1 
        copy = 1));

Next, let's create an array of values for the share price at expiration. Make a vector of perturbation values $\beta_{i}\in\beta$, which we use to perturb the initial share price value $S_{\circ}$.
* Create a perturbation vector $\beta_{i}\in\beta$ which holds values $\beta_{i}\in\left[0.8.1.2\right]$. We'll use these values to perturb or compute various things, e.g., possible share prices at expiration for `MU`:

In [9]:
β = range(0.5,stop=1.5,length=1001) |> collect;

Now, create an array holding possible share prices at expiration by perturbing the initial share price. 
* Store these values in the `share_price_array` variable. For compactness, let's use the [Array comprehension functionality provided by Julia](https://docs.julialang.org/en/v1/manual/arrays/#man-comprehensions) to populate `share_price_array`.

In [10]:
share_price_array = [βᵢ*Sₒ for βᵢ ∈ β];

Finally, let's compute the payoff and profit for the short strangle. We compute the payoff by calling the `payoff(...)` function, while the profit 

In [11]:
V = profit([put_contract_model, call_contract_model], share_price_array);

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.